In [37]:
# from pyDataverse documentation: https://pydataverse.readthedocs.io/en/latest/

# this code is used to install pydataverse within jupyter notebook
# import sys
# !{sys.executable} -m pip install pydataverse

import pandas as pd
#from pyDataverse.api import Api
from pyDataverse.api import NativeApi
from pyDataverse.models import Dataverse
import os  # this is so you can grep the current directory
import pprint
dirpath = os.getcwd()
print("current directory: " + dirpath)
os.chdir(dirpath)

current directory: D:\work-related\pyDataverse


In [38]:
# establish connection
# class Api(base_url, api_token=None, api_version='v1')
base_url = 'https://dataverse.ucla.edu/'
dv_token = '4eae5f4a-1a86-42a1-9764-cbebc0479a31'
#api = Api(base_url, dv_token)
api = NativeApi(base_url, dv_token)
#api.status

In [39]:
# get dataverse
#dv = 'ucla'
#resp = api.get_dataverse(dv)
#resp.json()['data']['creationDate']
dv = 'ucla'
resp = api.get_dataverse(dv)
resp.json()['data']['creationDate']

'2018-09-28T21:46:11Z'

In [40]:
# get dataset of the lariac4 dataset
identifier = 'doi:10.25346/S6/T4LHZF' 
print("identifier: ", identifier)
resp = api.get_dataset(identifier)
datasetID = resp.json()['data']['publisher']
print("publisher: ", datasetID)
ds_title = resp.json()['data']['latestVersion']['metadataBlocks']['citation']['fields'][0]['value']
print(ds_title)

identifier:  doi:10.25346/S6/T4LHZF
publisher:  UCLA Dataverse
LARIAC4 Classified_Lidar


In [41]:
# get list of all datafiles in dataset
# The dataset id can be extracted from the response retrieved from the API 
#      which uses the persistent identifier (/api/datasets/:persistentId/?persistentId=$PERSISTENT_IDENTIFIER).
# curl https://dataverse.ucla.edu/api/datasets/21062/versions/3.0/files | jq
# check what version you are up to
# curl https://dataverse.ucla.edu/api/datasets/21062/versions | jq . > filename.json
# curl https://dataverse.ucla.edu/api/datasets/21062/versions | jq
# identifier = 'doi:10.25346/S6/T4LHZF'
# version = '11.0'
#resp = api.get_datafiles(identifier, version)
#print(version)
resp = api.get_dataset(identifier)
datasetID = resp.json()['data']['id']
fileOut = resp.json()['data']
dataset_name = resp.json()['data']['latestVersion']['files'][9]
print(dataset_name)
# print(fileOut['latestVersion']['files'])
print(len(fileOut))
print(type(fileOut))
#print(fileOut)
print(fileOut.keys()) 

{'description': 'LARIAC4', 'label': 'L4_6276_1842d.las', 'restricted': True, 'version': 1, 'datasetVersionId': 1268, 'dataFile': {'id': 22171, 'persistentId': 'doi:10.25346/S6/T4LHZF/EASEAT', 'pidURL': 'https://doi.org/10.25346/S6/T4LHZF/EASEAT', 'filename': 'L4_6276_1842d.las', 'contentType': 'application/octet-stream', 'filesize': 117988410, 'description': 'LARIAC4', 'storageIdentifier': 's3-dataverse-lariac-direct://dataverse-lariac-direct:176447636b2-cc4fdc06fbe0', 'rootDataFileId': -1, 'md5': '762868e8c658d932ffd9e06a73bf4472', 'checksum': {'type': 'MD5', 'value': '762868e8c658d932ffd9e06a73bf4472'}, 'creationDate': '2020-12-08'}}
9
<class 'dict'>
dict_keys(['id', 'identifier', 'persistentUrl', 'protocol', 'authority', 'publisher', 'publicationDate', 'storageIdentifier', 'latestVersion'])


In [42]:
#print(fileOut)

In [43]:
import json
# output json file
with open('fileout2dv.json', 'w') as outfile:
    json.dump(fileOut, outfile)

In [44]:
import pandas as pd
# output csv file - read from the previous json file
df = pd.read_json(r'fileout2.json')
df.to_csv (r'fileout2.csv')

In [45]:
# test downloading file 'bundle'
# /api/access/datafiles/$id1,$id2,...$idN
# pyDataverse:   get_datafile_bundle(identifier)
# identifier is Metadata Id
#fileList = 'doi:10.25346/S6/T4LHZF/C9IZ9T, doi:10.25346/S6/T4LHZF/X2XS2C, doi.org/10.25346/S6/T4LHZF/I2NO6V'
# get_datafile(identifier, is_pid=True)
#file_doi = 'doi:10.25346/S6/T4LHZF/HF0XYG'
#file_id = '22175'
#resp = api.get_datafile(file_id)
#resp.json()

In [46]:
# use pprint.pprint to get the structure of the metadata output, then pull out what's necessary for a csv file
#import pprint
dir(pprint)
print("Number of files: ", len('fileOut'))
print('file id:', fileOut['id'])
print('doi:',fileOut['persistentUrl'])
file_label = resp.json()['data']['latestVersion']['files'][1]['label']
print("file label: ", file_label)
file_id = resp.json()['data']['latestVersion']['files'][1]['dataFile']['id']
print(file_id)
file_URL = resp.json()['data']['latestVersion']['files'][1]['dataFile']["pidURL"]
file_doi = resp.json()['data']['latestVersion']['files'][1]['dataFile']["persistentId"]
print(file_label,", ",file_id, ", ", file_doi, ", ", file_URL)
#print("Individual file structure:")
#pprint.pprint(fileOut)

Number of files:  7
file id: 21062
doi: https://doi.org/10.25346/S6/T4LHZF
file label:  L4_6273_1836d.las
22178
L4_6273_1836d.las ,  22178 ,  doi:10.25346/S6/T4LHZF/HUVC5T ,  https://doi.org/10.25346/S6/T4LHZF/HUVC5T


In [47]:
from datetime import date
total_files = (len(resp.json()['data']['latestVersion']['files']))
print(total_files, " files")
n = 0
today = date.today()
print("Today's date:", today)
filename1 = datetime.datetime.now().strftime("%Y%m%d-%H%M%S") 
filename1 = 'filelist-' + filename1+'.csv'
print(filename1)

f = open(filename1, 'w')
while  n <= (total_files-1):
    #print(n)
    file_label = resp.json()['data']['latestVersion']['files'][n]['label']
    #print("file label: ", file_label)
    file_id = resp.json()['data']['latestVersion']['files'][n]['dataFile']['id']
    #print(file_id)
    file_URL = resp.json()['data']['latestVersion']['files'][n]['dataFile']["pidURL"]
    #print(file_URL)
    file_doi = resp.json()['data']['latestVersion']['files'][n]['dataFile']["persistentId"]
    #print(file_doi)
    #line=""
    #line=n,file_label, file_id, file_doi ,file_URL
    #','.join(str(line))
    #line = str(n + file_label + file_id + file_doi + file_URL)
    #print(line)
    print(str(n) + ", " + file_label + ", " + str(file_id) + ", " + file_doi + ", " + file_URL)
    #f.write(str(line))
    f.write(str(n) + ", " + file_label + ", " + str(file_id) + ", " + file_doi + ", " + file_URL)
    f.write("\n")
    n = n+1
f.close()

59  files
Today's date: 2021-03-29
filelist-20210329-135407.csv
0, L4_6273_1836c.las, 22177, doi:10.25346/S6/T4LHZF/JTFKYM, https://doi.org/10.25346/S6/T4LHZF/JTFKYM
1, L4_6273_1836d.las, 22178, doi:10.25346/S6/T4LHZF/HUVC5T, https://doi.org/10.25346/S6/T4LHZF/HUVC5T
2, L4_6276_1836a.las, 23408, doi:10.25346/S6/T4LHZF/AYVINK, https://doi.org/10.25346/S6/T4LHZF/AYVINK
3, L4_6276_1836b.las, 23407, doi:10.25346/S6/T4LHZF/PZXZMJ, https://doi.org/10.25346/S6/T4LHZF/PZXZMJ
4, L4_6276_1836c.las, 23405, doi:10.25346/S6/T4LHZF/YXQN5B, https://doi.org/10.25346/S6/T4LHZF/YXQN5B
5, L4_6276_1836d.las, 23406, doi:10.25346/S6/T4LHZF/D6MBA1, https://doi.org/10.25346/S6/T4LHZF/D6MBA1
6, L4_6276_1842a.las, 22169, doi:10.25346/S6/T4LHZF/3J75IE, https://doi.org/10.25346/S6/T4LHZF/3J75IE
7, L4_6276_1842b.las, 22168, doi:10.25346/S6/T4LHZF/5QUJW4, https://doi.org/10.25346/S6/T4LHZF/5QUJW4
8, L4_6276_1842c.las, 22170, doi:10.25346/S6/T4LHZF/I2NO6V, https://doi.org/10.25346/S6/T4LHZF/I2NO6V
9, L4_6276_1842d.l

In [48]:
# eventually concat this to the output file to differentiate
from datetime import date
today = date.today()
print("Today's date:", today)
filename1 = datetime.datetime.now().strftime("%Y%m%d-%H%M%S") 
filename1 = 'filelist-' + filename1+'.csv'
print(filename1)

Today's date: 2021-03-29
filelist-20210329-135408.csv
